Algorithm

1. initial (manual) marks detection on zero frame
2. calc affin transform matrix for zero frame
3. saving marks and affin transform matrix for zero frame
4. i = 1

-----

5. marks detection on frame[i] using information about frame[i-1]
6. calc affin transform matrix on frame[i] using information about frame[i-1]
7. saving marks and affin transform matrix for frame[i]
8. i++
9. to step 5, until last frame

# Initial detection

In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
from os.path import join as pjoin
from tqdm import tqdm
import os

# Frame by frame detection